# Parsing Julia to Tapenade's IL

This notebook shows how to parse julia code into the format used by tapenade in a way to start developping source transform adjoint code. 

## Load Tapenade's tree structure
Start with compiling

In [4]:
#= 
Add some info and some calls to the makefile implemented in this example
=#

Load the newly created library

In [3]:
using Libdl
c_libname = "./treelib.so"
lib = Libdl.dlopen(c_libname) # Open the library explicitly.

Ptr{Nothing} @0x00000000034cd2f0

Define function handler to useful functions.

In [5]:
c_newTreeBuilder = Libdl.dlsym(lib, :newTreeBuilder)   # Get a symbol for the function to call.
c_startTree = Libdl.dlsym(lib, :startTree)
c_turnListFrontier = Libdl.dlsym(lib, :turnListFrontier)
c_startAnnotation = Libdl.dlsym(lib, :startTree) ### This is an extern function, and we might want to have this pointing to another lib!
c_putValue = Libdl.dlsym(lib, :putValue) ## Also extern
c_putTree = Libdl.dlsym(lib, :putTree) ## Also extern
c_oneLessWaiting = Libdl.dlsym(lib, :oneLessWaiting) 
c_putListTree = Libdl.dlsym(lib, :putListTree) ## Extern
c_startDummyTree = Libdl.dlsym(lib, :startDummyTree) ## Extern
c_terminateListTree = Libdl.dlsym(lib, :terminateListTree) ## Extern
c_getTreeBuilt = Libdl.dlsym(lib, :getTreeBuilt) ## Extern
c_getListTreeBuilt = Libdl.dlsym(lib, :getListTreeBuilt) ## Extern
c_deleteTreeBuilder = Libdl.dlsym(lib, :deleteTreeBuilder) ## Extern
c_resetTreeBuilder = Libdl.dlsym(lib, :resetTreeBuilder) ## Extern
c_removeTree = Libdl.dlsym(lib, :removeTree)
c_showTreeBuilderState = Libdl.dlsym(lib, :showTreeBuilderState) ## Debug purposes

Ptr{Nothing} @0x00007f824b783bfd

In [58]:
using Pkg
Pkg.add("DataStructures")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `/home/jeboucho/.julia/environments/v1.9/Project.toml`
  [864edb3b] + DataStructures v0.18.20
  No Changes to `/home/jeboucho/.julia/environments/v1.9/Manifest.toml`


In [59]:
using DataStructures

In [60]:
ct_testfile = "ct_test.jl"
orig_testfile = "kepler.jl"
basic_test = "testfile.jl"

"testfile.jl"

In [97]:
EndOfList = :EndOfList
cstSymb = :cst

:cst

In [98]:
typeof(EndOfList)

Symbol

In [113]:
function file2ast(fname::AbstractString, outfile::AbstractString)
    all_symbols = Set()
    open(outfile, "w") do out 
        code = read(fname, String)
        println(code)

        exp_in_waiting = Stack{Any}()
        
        prev = 0
        next = 1
        while next > prev 
            prev = next
            exp, next = Meta.parse(code, prev)
            push!(exp_in_waiting, exp)
            ## println("     ***** Currently looking at: ")
            ## println(exp)
            while !isempty(exp_in_waiting)
                e = pop!(exp_in_waiting)
                if e isa Expr 
                    # Parse expression
                    # Write current instruction
                    push!(all_symbols, e.head) 
                    println("     ***** Currently looking at: ")
                    println(e.head)
                    println(e.args)
                    write(out, e.head) # New C node from tree builder
                    write(out, "\n")
                    # If Current instruction is a call, make sure to add an end of list
                    # Same thing holds for a 'using' statement
                    #####################
                    # NOTE:             #
                    # One might need    #
                    # to check if other #
                    # keywords need this#
                    #####################
                    if e.head in (:call, :tuple, :using, :vect) 
                        push!(exp_in_waiting, EndOfList)
                    end
                    for arg in Iterators.reverse(e.args)
                        push!(exp_in_waiting, arg)
                    end
                elseif e isa Symbol
                    # Add symbol to file
                    write(out, e) # New leaf for current operator
                    write(out, "\n")
                else
                    # Neither a symbol nor an expression, it's probably a constant
                    write(out, cstSymb)
                    write(out, "\n")
                    ## println("Looking at cst things")
                    ## println(e)
                    write(out, string(e))
                    write(out, "\n")
                end
            end
                    
            
        end
    end
    return all_symbols
end

file2ast (generic function with 1 method)

In [114]:
all_symbols = file2ast(basic_test, "outputtest")

# testfile.jl
# No guarantee that this does anything meaningful, but if has some basic Julia things in it, with some control

# Import some modules
using LinearAlgebra, ForwardDiff

# Some basic definitions (including unicode characters, array, scientific notations...)
Tmax = 6                                   # maximum thrust
cTmax = (3600^2) / 1e6                     # conversion from Newtons to kg . Mm / h²
mass0 = 1500                               # initial mass of the spacecraft
β = 1.42e-02                               # engine specific impulsion
μ = 5165.8620912                           # Earth gravitation constant
t0 = 0                                     # initial time (final time is free)
x0 = [ 11.625, 0.75, 0, 6.12e-02, 0, π ]   # initial state (fixed initial longitude)
xf_fixed = [ 42.165, 0, 0, 0, 0 ]          # final state (free final longitude)

# More involved data structures
init = Dict{Real, Tuple{Real, Vector{Real}}}()

# Multiple things on a single line
tf = 1

Set{Any} with 13 elements:
  :vect
  :vcat
  :block
  :return
  :tuple
  :.
  :ref
  :curly
  :call
  :function
  :(=)
  :using
  :toplevel